In [1]:
import rpyc
import numpy as np

In [2]:
c = rpyc.connect("localhost", 18861)
c.root

In [4]:
c.root.process([10, 10])

[100, 100]

In [5]:
c.close()

In [2]:
import rpyc
import numpy as np
from computer_utils import SimpleSplitter, SimpleMerger, Overseer
from multiprocessing import Queue, Manager

rpyc.core.protocol.DEFAULT_CONFIG['allow_pickle'] = True


class Computer:
    
    def __init__(self, data, weight):
        self.data = data
        self.object_queue = Queue()
        self.weight = weight
        
        self.conns = {}
        
#         self.pinger = SimplePinger(self.conns)
        self.splitter = SimpleSplitter()
        self.merger = SimpleMerger()
        
    def add_connection(self, ip = 'localhost', port = 18861):
        self.conns[f'{ip}:{port}'] = rpyc.connect(ip, port)
        
    def compute(self):
        # creating a queue for data storage
        queue = Queue()
        sample_num = self.splitter.split(self.data, queue)
        # creating Overseers for the Processors
        ovs = Overseer(list(self.conns.values())[0])
        # Manager for the shared resources
        manager = Manager()
        
        res = manager.dict()
        # initializing task completeness list
        shared_status = manager.list([0] * sample_num)
        # created added task list to the Overseer
        res = ovs.put(queue, self.weight, res, shared_status)
        # merging the resuts 
        result = self.merger.merge(res)
        
        # closing connections
        for c in self.conns.values():
            c.close()
        
        return result
    
if __name__ == "__main__":
    data = np.arange(4).reshape(-1, 1)
    print('Initial data:', data, sep = '\n')
    cmp = Computer(data, 10)
    cmp.add_connection()
    print('Resulting data:', cmp.compute(), sep = '\n')

Initial data:
[[0]
 [1]
 [2]
 [3]]
Resulting data:
[[ 0]
 [10]
 [20]
 [30]]


```
import rpyc
import numpy as np
from computer_utils import SimpleSplitter, SimpleMerger, Overseer
from multiprocessing import Queue, Manager

rpyc.core.protocol.DEFAULT_CONFIG['allow_pickle'] = True


class Computer:
    
    def __init__(self, data, weight):
        self.data = data
        self.object_queue = Queue()
        self.weight = weight
        
        self.conns = {}
        
#         self.pinger = SimplePinger(self.conns)
        self.splitter = SimpleSplitter()
        self.merger = SimpleMerger()
        
    def add_connection(self, ip = 'localhost', port = 18861):
        self.conns[f'{ip}:{port}'] = rpyc.connect(ip, port)
        
    def compute(self):
        # creating a queue for data storage
        queue = Queue()
        sample_num = self.splitter.split(self.data, queue)
        # creating Overseers for the Processors
        self.ovs = [Overseer(conn) for conn in self.conns.values()]
        print(self.ovs)
        return None
        # Manager for the shared resources
        manager = Manager()
        
        res = manager.dict()
        # initializing task completeness list
        shared_status = manager.list([0] * sample_num)
        # created added task list to the Overseer
        res = ovs.put(queue, self.weight, res, shared_status)
        # merging the resuts
        result = self.merger.merge(res)
        
        # closing connections
        for c in self.conns:
            c.close()
        
        return result
    
if __name__ == "__main__":
    data = np.arange(4).reshape(-1, 1)
    print('Initial data:', data, sep = '\n')
    cmp = Computer(data, 10)
    cmp.add_connection()
    print('Resulting data:', cmp.compute(), sep = '\n')
```

In [2]:
data = np.ones((2, 1))

In [3]:
data

array([[1.],
       [1.]])

In [4]:
cmp = Computer(data, 10)

In [5]:
cmp.add_connection()

In [6]:
cmp.process()

TypeError: put() missing 1 required positional argument: 'samples'

In [10]:
len({1: 5, 2: 3}.items())

2